In [1]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\My_Files\\code\\Text-Summarizer'

## Entity

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



## Configuration Manager

In [7]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager():
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

## Components

In [10]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} Downloaded Data!!!   [INFO] : \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Pipeline

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-03 12:39:30,782: INFO: common: yaml file:config\config.yaml loaded successfully!]
[2023-07-03 12:39:30,785: INFO: common: yaml file:params.yaml loaded successfully!]
[2023-07-03 12:39:30,787: INFO: common: created directory at: artifacts]
[2023-07-03 12:39:30,789: INFO: common: created directory at: artifacts/data_ingestion]


[2023-07-03 12:39:34,364: INFO: 223447639: artifacts/data_ingestion/data.zip Downloaded Data!!!   [INFO] : 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D352:6A5D:256D93:3811A2:64A2742B
Accept-Ranges: bytes
Date: Mon, 03 Jul 2023 07:09:32 GMT
Via: 1.1 varnish
X-Served-By: cache-del21734-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1688368172.581348,VS0,VE714
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: c9aeae011900908b48822f2140e665ebb4c87529
Expires: Mon, 03 Jul 2023 07:14:32 GMT
Source-Age: 0

]
